# Kats 101 - Basics

Kats (**K**its to **A**nalyze **T**ime **S**eries) is a light-weight, easy-to-use, extenable, and generalizable framework to perform time series analysis in Python.  Time series analysis is an essential component of data science and engineering work.  Kats aims to provide a one-stop shop for techniques for univariate and multivariate time series including:

1. Forecasting  
2. Anomaly and Change Point Detection  
3. Feature Extraction 


and after introducing the basic Kats data structure, this Kats 101 notebook provides a basic introduction to each of these time series techniques in Kats.  The complete table of contents for Kats 101 is as follows: 

1. Kats Basics          
    1.1 Initiate `TimeSeriesData` Object         
    1.2 `TimeSeriesData` built-in operations         
2. Forecasting with Kats         
    2.1 An example with Prophet model         
    2.2 An example with Theta model         
3. Detection with Kats         
    3.1 What are the algorithms?         
    3.2 An example with outlier detection method         
    3.3 An example with CUSUM algorithm         
4. Feature extraction with Kats         
5. Summary         

**Note:** We provide two types of tutorial notebooks
- **Kats 101**, basic data structure and functionalities in Kats (this tutorial)  
- **Kats 20x**, advanced topics, including advanced forecasting techniques, advanced detection algorithms, `TsFeatures`, meta-learning, etc. 

# 1. Kats Basics
`TimeSeriesData` is the basic data structure in Kats to represented univariate and multivariate time series.  There are two ways to initiate it, henceforth referred to as "Method 1" and "Method 2":

1) `TimeSeriesData(df)`, where `df` is a `pd.DataFrame` object with a "time" column and any number of value columns.

2) `TimeSeriesData(time, value)`, where `time` is either a `pd.Series` or `pd.DatetimeIndex` object and `value` is either a `pd.Series` (for univariate) or a `pd.DataFrame` (for multivariate)

## 1.1 Initiate `TimeSeriesData` object
We will use the `air_passenger` and `multi_ts` datasets to demonstrate how to create a `TimeSeriesData` object for univariate and multivariate time series, respectively.

In [ ]:
%%capture
# For Google Colab:
!pip install kats
!wget https://raw.githubusercontent.com/facebookresearch/Kats/main/kats/data/air_passengers.csv
!wget https://raw.githubusercontent.com/facebookresearch/Kats/main/kats/data/multi_ts.csv

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

from kats.consts import TimeSeriesData

In [ ]:
try: # If running on Jupyter
    air_passengers_df = pd.read_csv("../kats/data/air_passengers.csv")
except FileNotFoundError: # If running on colab
    air_passengers_df = pd.read_csv("air_passengers.csv")

"""
Note: If the column holding the time values is not called "time", you will want to specify 
the name of this column using the time_col_name parameter in the TimeSeriesData constructor.
"""
air_passengers_df.columns = ["time", "value"]
air_passengers_df.head()

In [ ]:
try: # If running on Jupyter
    multi_ts_df = pd.read_csv("../kats/data/multi_ts.csv", index_col=0)
except FileNotFoundError: # If running on colab
    multi_ts_df = pd.read_csv("multi_ts.csv", index_col=0)
multi_ts_df.columns = ["time", "v1", "v2"]
multi_ts_df.head()

Here we construct `TimeSeriesData` objects for each time series using Method 1.

In [ ]:
air_passengers_ts = TimeSeriesData(air_passengers_df)
multi_ts = TimeSeriesData(multi_ts_df)

In [ ]:
# check that the type of the data is a "TimeSeriesData" object for both cases
print(type(air_passengers_ts))
print(type(multi_ts))

In [ ]:
# For the air_passengers TimeSeriesData, check that both time and value are pd.Series
print(type(air_passengers_ts.time))
print(type(air_passengers_ts.value))

In [ ]:
# For the multi_ts TimeSeriesData, time is a pd.Series and value is a pd.DataFrame
print(type(multi_ts.time))
print(type(multi_ts.value))

Now, we show how to construct the same `TimeSeriesData` objects as before using Method 2.

In [ ]:
air_passengers_ts_from_series = TimeSeriesData(time=air_passengers_df.time, value=air_passengers_df.value)
multi_ts_from_series = TimeSeriesData(time=multi_ts_df.time, value=multi_ts_df[['v1', 'v2']])

`TimeSeriesData` can accomodate time expressed as a variety of different types, including 
- standard `datetime`, 
- `pandas.Timestamp`,
- a `str` (if in a non-standard format or if efficiency is important, use the optional `date_format` argument),
- `int` (i.e. unix time).

Here is an example of how to construct a `TimeSeriesData` object when time is provided in unix time format.


Here's an example where the time is auto-interpreted from a unix time format. Using this format just requires a couple optional parameters in the `TimeSeriesData` constructor:
- `use_unix_time = True`
- `unix_time_units="s"` (the default is `"ns"`, indicating nanoseconds)

In [ ]:
from dateutil import parser
from datetime import datetime

# Convert time from air_passengers data to unix time
air_passengers_ts_unixtime = air_passengers_df.time.apply(
    lambda x: datetime.timestamp(parser.parse(x))
)

air_passengers_ts_unixtime

In [ ]:
# Create the TimeSeriesData object with the extra arguments to accomodate unix time 
ts_from_unixtime = TimeSeriesData(
        time=air_passengers_ts_unixtime, 
        value=air_passengers_df.value, 
        use_unix_time=True, 
        unix_time_units="s"
)

ts_from_unixtime

## 1.2 `TimeSeriesData` built-in operations

The `TimeSeriesData` object supports many of the same operations of the standard `pandas.DataFrame`, including:
- Slicing
- Math Operations
- Extend 
- Plotting
- Utility Functions (`to_dataframe`, `to_array`, `is_empty`, `is_univariate`)

We give examples of each as follows.

## Slicing

In [ ]:
air_passengers_ts[1:5]

## Math operations

In [ ]:
air_passengers_ts[1:5] + air_passengers_ts[1:5]

In [ ]:
# Equality and Inequality are also supported:

air_passengers_ts == air_passengers_ts_from_series

In [ ]:
multi_ts == multi_ts_from_series

In [ ]:
# length

len(air_passengers_ts)

## Extend

In [ ]:
# Creating two slices
ts_1 = air_passengers_ts[0:3]
ts_2 = air_passengers_ts[3:7]

ts_1.extend(ts_2)
ts_1

## Plotting

In [ ]:
%matplotlib inline

# Must pass the name of the value columns to plot
air_passengers_ts.plot(cols=['value'])
plt.show()

In [ ]:
# We can plot multiple time series from multi_ts by passing in the name of each value column we want to plot
multi_ts.plot(cols=['v1','v2'])
plt.show()

## Utility functions

Here we provide examples of a few useful Kats utility functions for `TimeSeriesData`.  They can be helpful for working with external libraries.

### Convert to `pandas.DataFrame`

In [ ]:
air_passengers_ts.to_dataframe().head()

### Convert to `numpy.ndarray`

In [ ]:
air_passengers_ts.to_array()[0:5]

### Check basic characteristics of the time series

In [ ]:
air_passengers_ts.is_empty()

In [ ]:
air_passengers_ts.is_univariate()

In [ ]:
multi_ts.is_univariate()

# 2. Forecasting with Kats

We currently support the following 10 base forecasting models: 

1. Linear  
2. Quadratic   
3. ARIMA   
4. SARIMA   
5. Holt-Winters   
6. Prophet   
7. AR-Net   
8. LSTM   
9. Theta   
10. VAR   


Each models follows the `sklearn` model API pattern:  we create an instance of the model class and then call its `fit` and `predict` methods.  In this section, we provide examples for the Prophet and Theta models.  A more in-depth introduction to forecasting in Kats is provided in the Kats 201 tutorial.



## 2.1 An example with Prophet model

We will demonstrate how to use Prophet model to forecast with the `air_passengers` data set. Note: this example requires that fbprophet be installed (for example, `pip install kats[prophet]` or `pip install kats[all]`)

In [ ]:
# import the param and model classes for Prophet model
from kats.models.prophet import ProphetModel, ProphetParams

# create a model param instance
params = ProphetParams(seasonality_mode='multiplicative') # additive mode gives worse results

# create a prophet model instance
m = ProphetModel(air_passengers_ts, params)

# fit model simply by calling m.fit()
m.fit()

# make prediction for next 30 month
fcst = m.predict(steps=30, freq="MS")

In [ ]:
# the predict method returns a dataframe as follows
fcst.head()

In [ ]:
# visualize the results with uncertainty intervals
m.plot()

## 2.2 An example with Theta model


We will now use the Theta model to forecast with the `air_passengers` data set.  

The Theta Method (Assimakopoulos and Nikolopoulos, 2000) is a univariate forecasting method that fits two Theta lines: 1) a linear interpolation (called the `Theta=0`) and 2) a second-order difference (called the `Theta=2` line), and then combines them to build a forecast.  Prior to running this forecast, we test the time series for seasonality, deseaonalize if seasonality is detected, and then reseasonalize the calculated forecasts.  

Hyndman and Billah (2003) showed that the Theta Method is equivalent to simple exponential smoothing with drift.  In Kats we use this underlying model to calculate prediction intervals for `ThetaModel`.

Our implementation of `ThetaModel` in Kats is similar to the [thetaf function in R](https://pkg.robjhyndman.com/forecast/reference/thetaf.html).  Because each of our time series models follow the `sklearn` model API pattern, the code using `ThetaModel` is quite similar to the example above using 
`ProphetModel`: we initialize the model with its parameters and then call the `fit` and `predict` methods.  We can then use the `plot` method to visualize our forecast.

In [ ]:
# import param and model from `kats.models.theta`
from kats.models.theta import ThetaModel, ThetaParams

# create ThetaParam with specifying seasonality param value
params = ThetaParams(m=12)

# create ThetaModel with given data and parameter class
m = ThetaModel(data=air_passengers_ts, params=params)

# call fit method to fit model
m.fit()

# call predict method to predict the next 30 steps
res = m.predict(steps=30, alpha=0.2)

# visualize the results
m.plot()

# 3. Detection with Kats

Kats provides a set of models and algorithms to detect outliers, change points, and trend changes in time series data.


## 3.1 What are the algorithms?

To detect a specific pattern, we provided different algorithms, which is summarized as follows.
- **Outlier Detection**. This usually refers to a abnormal spike in a time series data, which can be detected with `OutlierDetector`
- **Change Point Detection**. This refers to a sudden change that the time series have different statistical properties before and after the change. We provided three major algorithms to detect such patterns:
    - CUSUM Detection
    - Bayesian Online Change Point Detection (BOCPD)
    - Stat Sig Detection
- **Trend Change Detection**. This refers to a slow trend change on the time series data, which can be detected with Mann-Kendall detection algorithm, `MKDetector`

In this tutorial, we will demonstrate the usage of two detectors: `OutlierDetector` and `CUSUM`.  A more in-depth introduction to detection in Kats is provided in the Kats 202 tutorial.



## 3.2 An example with outlier detection method

We provide the `OutlierDetector` module to detect outliers in time series.  Since outliers can cause so many problems in downstream processing, it is important to be able to detect them.  `OutlierDetector` also provides functionality to handle or remove outliers once they are found.

Our outlier detection algorithm works as follows:

- We do a [seasonal decomposition](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html) of the input time series with additive or multiplicative decomposition as specified (default is additive)
- We generate a residual time series by either removing only trend or both trend and seasonality if the seasonality is strong.
- We detect points in the residual which are outside 3 times the inter quartile range.  This multiplier can be tuned using the `iqr_mult` parameter in `OutlierDetector`.

Our example below copies the `air_passengers` data set and manually inserts outliers into it. We then use `OutlierDetector` to find them. 

In [ ]:
# deep copy the air_passenger_df 
air_passengers_outlier_df = air_passengers_df.copy(deep=True)

# manually add outlier on the date of '1950-12-01'
air_passengers_outlier_df.loc[air_passengers_outlier_df.time == '1950-12-01','value']*=5
# manually add outlier on the date of '1959-12-01'
air_passengers_outlier_df.loc[air_passengers_outlier_df.time == '1959-12-01', 'value']*=4

In [ ]:
# visualize the raw data
air_passengers_outlier_df.plot(x='time', y='value', figsize=(15,8))
plt.show()

In [ ]:
# transform the outlier data into `TimeSeriesData` Object
air_passengers_outlier_ts = TimeSeriesData(air_passengers_outlier_df)

In [ ]:
from kats.detectors.outlier import OutlierDetector

ts_outlierDetection = OutlierDetector(air_passengers_outlier_ts, 'additive') # call OutlierDetector
ts_outlierDetection.detector() # apply OutlierDetector

Here we look at the outliers that the algorithm found.

In [ ]:
ts_outlierDetection.outliers[0]

After detecting the outlier, we can now easily removal them from the data. Here we will explore two options: 
- **No Interpolation**: outlier data points will be replaced with **NaN** values
- **With Interpolation**: outlier data points will be replaced with **linear interploation** values

In [ ]:
air_passengers_ts_outliers_removed = ts_outlierDetection.remover(interpolate = False) # No interpolation
air_passengers_ts_outliers_interpolated = ts_outlierDetection.remover(interpolate = True) # With interpolation

Here we visualize the difference between these two approaches to removing outliers.

In [ ]:
fig, ax = plt.subplots(figsize=(20,8), nrows=1, ncols=2)

air_passengers_ts_outliers_removed.to_dataframe().plot(x = 'time',y = 'y_0', ax= ax[0])
ax[0].set_title("Outliers Removed : No interpolation")
air_passengers_ts_outliers_interpolated.to_dataframe().plot(x = 'time',y = 'y_0', ax= ax[1])
ax[1].set_title("Outliers Removed : With interpolation")
plt.show()

## 3.3 An example with CUSUM algorithm

Cusum is a method to detect an up/down shift of means in a time series. Our implementation has two main steps:

1.  **Locate the change point:** This is an iterative process where we initialize a change point (in the middle of the time series) and CUSUM time series based on this change point.  The next changepoint is the location where the previous CUSUM time series is maximized (or minimized).  This iteration continues until either 1) a stable changepoint is found or 2) we exceed the limit number of iterations.

2.  **Test the change point for statistical significance:** Conduct log likelihood ratio test to test if the mean of the time series changes at the changepoint calculated in Step 1.  The null hypothesis is that there is no change in mean.

By default, we report a detected changepoint if and only if we reject the null hypothesis in Step 2.  

Here are a few additional points worth mentioning:
- We assume there is at most one increase change point and at most one decrease change point.  You can use the `change_directions` argument in the detector to specify whether you are looking an increase, a decrease, or both (default is both).
- We use Gaussian distribution as the underlying model to calculate the CUSUM time series value and conduct the hypothesis test.

In [ ]:
# import packages
from kats.consts import TimeSeriesData, TimeSeriesIterator
from kats.detectors.cusum_detection import CUSUMDetector

# synthesize data with simulation
np.random.seed(10)
df = pd.DataFrame(
    {
        'time': pd.date_range('2019-01-01', '2019-03-01'),
        'increase':np.concatenate([np.random.normal(1,0.2,30), np.random.normal(2,0.2,30)]),
        'decrease':np.concatenate([np.random.normal(1,0.3,50), np.random.normal(0.5,0.3,10)]),
    }
)

It is straightforward to use `CUSUMDetector` to detect an increase or a decrease when there is one.

In [ ]:
# detect increase
timeseries = TimeSeriesData(
    df.loc[:,['time','increase']]
)
detector = CUSUMDetector(timeseries)

# run detector
change_points = detector.detector(change_directions=["increase"])

# plot the results
detector.plot(change_points)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# detect decrease
timeseries = TimeSeriesData(
    df.loc[:,['time','decrease']]
)
detector = CUSUMDetector(timeseries)

# run detector
change_points = detector.detector(change_directions=["decrease"])

# plot the results
detector.plot(change_points)
plt.xticks(rotation=45)
plt.show()

If we try to detect a decrease in a series where there is only an increase, no change point will be found.

In [ ]:
# detect increase
timeseries = TimeSeriesData(
    df.loc[:,['time','increase']]
)
detector = CUSUMDetector(timeseries)

# run detector
change_points = detector.detector(change_directions=["decrease"])

# plot the results
detector.plot(change_points)
plt.xticks(rotation=45)
plt.show()

If we do not specify which change directions we are looking for, `CUSUMDetector` will look for both increases and decreases.  In the case below where there is only an increase, it will detect that increase.

In [ ]:
# detect increase
timeseries = TimeSeriesData(
    df.loc[:,['time','increase']]
)
detector = CUSUMDetector(timeseries)

# run detector
change_points = detector.detector()

# plot the results
detector.plot(change_points)
plt.xticks(rotation=45)
plt.show()

# 4. Feature Extraction with Kats

We provide the `TsFeatures` module to calculate a set of meaningful features for a time series, including:
- STL (Seasonal and Trend decomposition using Loess) Features
    - Strength of Seasonality
    - Strength of Trend
    - Spikiness
    - Linearity
- Amount of Level Shift
- Presence of Flat Segments
- ACF and PACF Features
- Hurst Exponent
- ARCH Statistic

Given a collection of time series, these features can be used to identify specific series that are similar or outlying.  Our `TsFeatures` module is similar to the one that is [freely available in R](https://pkg.robjhyndman.com/tsfeatures/index.html).

These features also play a crucial role in many downstream projects, including 
1. “Meta-learning”, i.e., choosing the best forecasting model based on characteristics of the input time series 
2. Time series classification and clustering analysis
3. Nowcasting algorithms for better short-term forecasting

Now we will show you how to use `TsFeatures` get the features for the `air_passenger` data set.

In [ ]:
# Initiate feature extraction class
from kats.tsfeatures.tsfeatures import TsFeatures
tsFeatures = TsFeatures()

In [ ]:
features_air_passengers = TsFeatures().transform(air_passengers_ts)

Now we can see the dictionary of features as follows.

In [ ]:
# features_air_passengers
features_air_passengers

# 5. Summary

in this tutorial, we have shown the basic operations of **Kats** in time series application, including basic introductions to forecasting, detection, feature extraction in Kats.

If you want to explore these topics in more detail, you can check out Kats 20x tutorials:

- [Kats 201 Forecasting with Kats](kats_201_forecasting.ipynb)
- [Kats 202 Detection with Kats](kats_202_detection.ipynb)
- [Kats 203 Time Series Features](kats_203_tsfeatures.ipynb)
- [Kats 204 Forecasting with Meta-Learning](kats_204_metalearning.ipynb)
- [Kats 205 Forecasting with Global Model](kats_205_globalmodel.ipynb)